In [ ]:
!pip install ipython-autotime --quiet
%load_ext autotime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.1 MB/s eta 0:00:00
time: 208 µs (started: 2025-07-10 14:21:20 +00:00)


time: 6.06 ms (started: 2025-07-10 14:21:20 +00:00)


In [ ]:
!pip install Bio py3Dmol rdkit-pypi gdown --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.5 MB/s eta 0:00:00
time: 13.5 s (started: 2025-07-10 14:21:20 +00:00)


In [ ]:
!pip install rdkit biopython pyyaml easydict tensorboard lmdb prody pypdb #  replaced python-lmdb with lmdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 64.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.7/299.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.8 MB/s eta 0:00:00
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Created wheel for prody: filename=prody-2.4.1-cp311-cp311-linux_x86_64.whl size=22369354 sha256=d45638afd3235d7e8bd1e6548f4a73124384cd5b1f5f2d1ca5903282760f58ba
  Stored in directory: /root/.cache/pip/wheels/6c/b1/85/e303653053b48dfb79b141ab7628dc80f0b7e345bafb881fc9


time: 50.5 s (started: 2025-07-10 14:21:33 +00:00)


In [ ]:
# !pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-geometric==2.3.1

Looking in indexes: https://download.pytorch.org/whl/cu118
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html


In [ ]:
# # PyTorch Geometric (Colab-specific build)
# !pip uninstall torch-scatter torch-sparse -y
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

In [ ]:
import os
import subprocess
import requests
import numpy as np
import py3Dmol
import torch

from Bio.PDB import PDBParser

In [ ]:
base_dir = "/content/"

## Identify PfEMP1's pockets (functional sites)

In [ ]:
def install_fpocket():
    """Clones and builds fpocket from GitHub."""
    if os.path.exists("fpocket"):
        print("fpocket already cloned.")
    else:
        print("Cloning fpocket repository...")
        subprocess.run(["git", "clone", "https://github.com/Discngine/fpocket.git"], check=True)

    print("Building fpocket...")
    subprocess.run(["make"], cwd="fpocket", check=True)

    # Add fpocket binary to PATH if needed
    fpocket_path = os.path.abspath("fpocket/bin")
    os.environ["PATH"] += os.pathsep + fpocket_path
    print(f"fpocket installed and added to PATH: {fpocket_path}")


def download_alphafold_structure(uniprot_id, output_path=f"{base_dir}PfEMP1.pdb"):
    url = f"https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb"
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded AlphaFold structure: {output_path}")
    else:
        raise Exception("Failed to download structure. Check UniProt ID.")


def run_fpocket(pdb_file):
    print(f"Running fpocket on {pdb_file}...")
    result = subprocess.run(["fpocket/bin/fpocket", "-f", pdb_file], capture_output=True, text=True)
    if result.returncode == 0:
        print("fpocket finished successfully.")
    else:
        print("fpocket failed:")
        print(result.stderr)


def list_pockets(pocket_dir=f"{base_dir}PfEMP1_out/pockets/"):
    if not os.path.exists(pocket_dir):
        print("Pocket directory not found.")
        return

    pockets = [f for f in os.listdir(pocket_dir) if f.endswith('.pdb')]
    print(f"🔎 Found {len(pockets)} pockets.")
    for i, p in enumerate(pockets[:5]):
        print(f"Pocket {i+1}: {p}")

In [ ]:
install_fpocket()

Cloning fpocket repository...
Building fpocket...
fpocket installed and added to PATH: /content/fpocket/bin


In [ ]:
uniprot_id = "Q8I3Y6"
pdb_file = f"{base_dir}PfEMP1.pdb"

download_alphafold_structure(uniprot_id, pdb_file)

Downloaded AlphaFold structure: /content/PfEMP1.pdb


In [ ]:
run_fpocket(pdb_file)

Running fpocket on /content/PfEMP1.pdb...
fpocket finished successfully.


In [ ]:
list_pockets()

🔎 Found 4 pockets.
Pocket 1: pocket1_atm.pdb
Pocket 2: pocket3_atm.pdb
Pocket 3: pocket4_atm.pdb
Pocket 4: pocket2_atm.pdb


In [ ]:
def get_pocket_centers(pockets_info_path=f"{base_dir}PfEMP1_out/PfEMP1_info.txt"):
    centers = {}
    with open(pockets_info_path, "r") as f:
        for line in f:
            if line.startswith("Pocket"):
                parts = line.strip().split()
                pocket_num = int(parts[1].replace(":", ""))
            if "Center" in line:
                coords = line.strip().split(":")[1].strip().split()
                center = tuple(map(float, coords))
                centers[pocket_num] = center
    return centers

def get_pocket_residues(pocket_pdb_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("pocket", pocket_pdb_path)
    residues = []
    for model in structure:
        for chain in model:
            for residue in chain:
                res_id = residue.get_id()
                res_name = residue.get_resname()
                residues.append((res_name, res_id[1]))  # (ALA, 123)
    return residues

def visualize_pocket_py3Dmol(protein_pdb_path, pocket_pdb_path):
    with open(protein_pdb_path, 'r') as f:
        protein_pdb = f.read()
    with open(pocket_pdb_path, 'r') as f:
        pocket_pdb = f.read()

    view = py3Dmol.view(width=800, height=600)
    view.addModel(protein_pdb, 'pdb')
    view.setStyle({'cartoon': {'color': 'lightblue'}})

    view.addModel(pocket_pdb, 'pdb')
    view.setStyle({'model': 1}, {'surface': {'opacity': 0.7, 'color': 'orange'}})

    view.zoomTo()
    return view.show()

In [ ]:
# visualize_pocket_py3Dmol("PfEMP1.pdb", f"{base_dir}PfEMP1_out/pockets/pocket1_atm.pdb")

In [ ]:
# visualize_pocket_py3Dmol("PfEMP1.pdb", f"{base_dir}PfEMP1_out/pockets/pocket2_atm.pdb")

In [ ]:
def get_all_pockets_data(pockets_dir=f"{base_dir}PfEMP1_out/pockets/"):
    pocket_data = {}

    if not os.path.exists(pockets_dir):
        raise FileNotFoundError(f"Directory not found: {pockets_dir}")

    pocket_files = sorted([f for f in os.listdir(pockets_dir) if f.endswith('_atm.pdb')])

    for pocket_file in pocket_files:
        pocket_path = os.path.join(pockets_dir, pocket_file)
        with open(pocket_path, 'r') as f:
            pdb_content = f.read()
            pocket_data[pocket_file] = pdb_content

    return pocket_data

In [ ]:
def get_pocket_center_from_pdb(pocket_pdb_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("pocket", pocket_pdb_path)

    coords = []
    for atom in structure.get_atoms():
        coords.append(atom.coord)

    if not coords:
        raise ValueError("No atoms found in pocket file.")

    center = np.mean(coords, axis=0)
    return tuple(center)

In [ ]:
def visualize_all_pockets(protein_pdb_path, pocket_data_dict):
    # Load protein structure
    with open(protein_pdb_path, 'r') as f:
        protein_pdb = f.read()

    for i, (pocket_name, pocket_pdb) in enumerate(pocket_data_dict.items(), start=1):
        view = py3Dmol.view(width=800, height=600)
        view.addModel(protein_pdb, 'pdb')  # Whole protein
        view.setStyle({'cartoon': {'color': 'lightblue'}})

        view.addModel(pocket_pdb, 'pdb')  # Pocket
        view.setStyle({'model': 1}, {'surface': {'opacity': 0.7, 'color': 'orange'}})

        view.zoomTo()
        print(f"Pocket {i}: {pocket_name}")
        view.show()

In [ ]:
pockets = get_all_pockets_data()

In [ ]:
centers = []

for pocket_file in pockets:
    center = get_pocket_center_from_pdb(f"{base_dir}PfEMP1_out/pockets/{pocket_file}")
    print(f"Pocket {pocket_file} center: {center}")
    centers.append(center)

print(centers)

Pocket pocket1_atm.pdb center: (-18.919567, -2.989, 13.69187)
Pocket pocket2_atm.pdb center: (10.693321, 2.3687997, -23.15884)
Pocket pocket3_atm.pdb center: (12.2715225, 3.3361301, -13.966915)
Pocket pocket4_atm.pdb center: (18.172617, 3.806857, -20.537235)
[(-18.919567, -2.989, 13.69187), (10.693321, 2.3687997, -23.15884), (12.2715225, 3.3361301, -13.966915), (18.172617, 3.806857, -20.537235)]


In [ ]:
for pocket_num in range(1, len(pockets)):
    residues = get_pocket_residues(f"{base_dir}PfEMP1_out/pockets/pocket{pocket_num}_atm.pdb")
    print(f"Pocket {pocket_num} has {len(residues)} residues. Sample:", residues[:5])

Pocket 1 has 7 residues. Sample: [('LYS', 97), ('LEU', 101), ('LYS', 100), ('ALA', 15), ('LEU', 16)]
Pocket 2 has 6 residues. Sample: [('SER', 67), ('ASP', 49), ('GLU', 65), ('LYS', 66), ('TYR', 70)]
Pocket 3 has 9 residues. Sample: [('LEU', 40), ('HIS', 54), ('CYS', 69), ('LEU', 53), ('GLU', 64)]


In [ ]:
visualize_all_pockets(f"{base_dir}PfEMP1.pdb", pockets)

Pocket 1: pocket1_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 2: pocket2_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 3: pocket3_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 4: pocket4_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
def visualize_single_pocket(protein_pdb_path, pocket_data_dict, pocket_index=1):
    pocket_items = list(pocket_data_dict.items())
    if pocket_index < 1 or pocket_index > len(pocket_items):
        raise ValueError("Invalid pocket index")

    pocket_name, pocket_pdb = pocket_items[pocket_index - 1]

    with open(protein_pdb_path, 'r') as f:
        protein_pdb = f.read()

    view = py3Dmol.view(width=800, height=600)
    view.addModel(protein_pdb, 'pdb')
    view.setStyle({'cartoon': {'color': 'lightblue'}})

    view.addModel(pocket_pdb, 'pdb')
    view.setStyle({'model': 1}, {'surface': {'opacity': 0.7, 'color': 'orange'}})

    view.zoomTo()
    print(f"Pocket {pocket_index}: {pocket_name}")
    # print(pocket_pdb)
    view.show()


    return view

In [ ]:
def save_py3dmol_to_html(view, output_file='structure.html'):
    html_str = view._make_html()
    with open(output_file, 'w') as f:
        f.write(html_str)

In [ ]:
for pocket_num in range(1, len(pockets) +1):
    view = visualize_single_pocket("PfEMP1.pdb", pockets, pocket_index=pocket_num)
    save_py3dmol_to_html(view, f"pocket_{pocket_num}.html")

Pocket 1: pocket1_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 2: pocket2_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 3: pocket3_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Pocket 4: pocket4_atm.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 2. Generate ligands

##### Inspire from https://colab.research.google.com/drive/113NKjh6wBZhOYEy6VZwd03zD3cJffogL?usp=sharing


In [ ]:
# !pip install ipython-autotime --quiet
# %load_ext autotime
# !pip install rdkit biopython pyyaml easydict tensorboard lmdb gdown prody pypdb --quiet #  replaced python-lmdb with lmdb

In [ ]:
# import torch
# print(f"torch version {torch.__version__}")
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html --quiet
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git  --quiet

#### Install Pocket2Mol

In [ ]:
!git clone https://github.com/pengxingang/Pocket2Mol --depth 1 --quiet

In [ ]:
# %cd /content/Pocket2Mol/data
# !test -f crossdocked_pocket10.tar.gz || gdown 10KGuj15mxOJ2FBsduun2Lggzx0yPreEU && tar -xzf crossdocked_pocket10.tar.gz
# !test -f split_by_name.pt || gdown 1mycOKpphVBQjxEbpn1AwdpQs8tNVbxKY
# !ls -l

In [ ]:
%cd /content/Pocket2Mol/ckpt
!test -f pretrained_Pocket2Mol.pt || gdown 1WaoEj9RDG4VEcyHEmgsjbh958txm1W6x
!ls -l

/content/Pocket2Mol/ckpt
Downloading...
From (original): https://drive.google.com/uc?id=1WaoEj9RDG4VEcyHEmgsjbh958txm1W6x
From (redirected): https://drive.google.com/uc?id=1WaoEj9RDG4VEcyHEmgsjbh958txm1W6x&confirm=t&uuid=c2f35825-facc-4498-86fd-c88670ab2f8d
To: /content/Pocket2Mol/ckpt/pretrained_Pocket2Mol.pt
100% 44.9M/44.9M [00:00<00:00, 112MB/s]
total 43876
-rw-r--r-- 1 root root 44921289 May 30  2022 pretrained_Pocket2Mol.pt
-rw-r--r-- 1 root root      309 Jul 10 14:31 README.md


In [ ]:
# # Clone repo and move in
# !git clone https://github.com/pengxingang/Pocket2Mol.git
# %cd Pocket2Mol

# # Install base dependencies
# !pip install -r requirements.txt

# # Install PyTorch Geometric (Colab-friendly)
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.1+cu118.html

# # Download checkpoint
# !gdown 1sK4LjgDNZ0TWK1gRjFePYGi9m9oy6iyd -O ckpt/pocket2mol.pth


In [ ]:
class LigandGenerator:
    def __init__(self, pdb_file, pocket_number, pocket_dir=f"{base_dir}PfEMP1_out/pockets/"):
        # self.parser = PDBParser(QUIET=True)
        # self.structure = self.parser.get_structure("protein", pdb_file)
        self.pdb_file = pdb_file
        self.pocket_dir = pocket_dir
        self.pocket_number = pocket_number
        self.pocket_path = os.path.join(pocket_dir, f"pocket{pocket_number}_atm.pdb")


    # Parse Pocket Volume from pockets_info.txt
    def get_pocket_volume(self, pockets_info_path=f"{base_dir}PfEMP1_out/pockets_info.txt"):
        with open(pockets_info_path, 'r') as f:
            lines = f.readlines()

        pocket_header = f"Pocket {self.pocket_number} :"
        for i, line in enumerate(lines):
            if line.strip().startswith(pocket_header):
                # Look ahead for Volume line
                for j in range(i+1, i+10):
                    if j < len(lines) and "Volume" in lines[j]:
                        volume_str = lines[j].split(":")[1].strip().split()[0]
                        return float(volume_str)
        return None


    # Map Volume → Max Molecular Weight
    def estimate_max_molwt_from_volume(self, volume):
        """
        Rough heuristic: 1 Å³ ≈ 1 Da, scaled with a safety margin.
        """
        return min(volume * 1.0, 600)  # Cap to ~600 Da to avoid very large ligands


    # Plug into Filtering Code
    def passes_filters(self, smiles, max_molwt, min_qed=0.4):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return False

        mw = Descriptors.MolWt(mol)
        qed_score = QED.qed(mol)
        hba = Descriptors.NumHAcceptors(mol)
        hbd = Descriptors.NumHDonors(mol)
        logp = Descriptors.MolLogP(mol)

        if (
            100 < mw <= max_molwt and
            qed_score >= min_qed and
            hba <= 10 and
            hbd <= 5 and
            logp <= 5
        ):
            return True
        return False


    # def generate_pocket_compatible_smiles(
    #     self,
    #     n=100,
    #     pockets_info_path="PfEMP1_out/pockets_info.txt",
    #     pocket_number=1,
    #     min_qed=0.4
    # ):
    #     print("Loading pretrained SMILES-VAE...")
    #     generator: DistributionLearningGenerator = load_generator('smiles_vae', 'guacamol_baselines')

    #     print("Extracting pocket volume...")
    #     volume = get_pocket_volume(pockets_info_path, pocket_number)
    #     if volume is None:
    #         raise ValueError(f"Could not find volume for Pocket {pocket_number}")
    #     max_molwt = estimate_max_molwt_from_volume(volume)

    #     print(f"Pocket {pocket_number} Volume: {volume:.2f} Å³ → Max MW: {max_molwt:.2f} Da")

    #     print(f"Generating {n} molecules...")
    #     raw_smiles = generator.generate_smiles(n)

    #     print("Filtering for pocket-compatible, drug-like molecules...")
    #     filtered = [smi for smi in raw_smiles if passes_filters(smi, max_molwt, min_qed)]

    #     print(f"{len(filtered)} molecules passed all filters.")
    #     return filtered


    def generate_smiles(self, max_ligands=10, config_path=f"{base_dir}pocket2mol_config.yml", checkpoint_path="", output_dir="outputs"):
        # Step 1: Get pocket center
        center = get_pocket_center_from_pdb(self.pocket_path)
        center_str = ",".join(f"{c:.2f}" for c in center)
        print(f"Pocket center: {center_str}")

        # Step 2: Ensure output directory exists
        os.makedirs(output_dir, exist_ok=True)

        # Step 3: Build and run command
        command = [
            "python", f"{base_dir}Pocket2Mol/sample_for_pdb.py",
            "--pdb_path", self.pdb_file,
            "--center", " " + center_str,
            "--outdir", output_dir,
            # "--num_return", str(max_ligands),
            "--config", config_path
            # "--checkpoint", checkpoint_path
        ]

        print("Running Pocket2Mol...")
        result = subprocess.run(command, capture_output=True, text=True)

        if result.returncode == 0:
            print("Pocket2Mol completed successfully.")
            return result
        else:
            print("Pocket2Mol failed:")
            print(result.stderr)

In [ ]:
ligand_generator = LigandGenerator(f"{base_dir}PfEMP1.pdb", pocket_number=1)

In [ ]:
%cd /content/Pocket2Mol

/content/Pocket2Mol


In [ ]:
volume = ligand_generator.get_pocket_volume(f"{base_dir}PfEMP1_out/PfEMP1_info.txt")
if volume is not None:
    print(f"Pocket 1 volume: {volume:.1f} Å³")
else:
    print("Volume not found for Pocket 1.")

max_mw = ligand_generator.estimate_max_molwt_from_volume(volume)
print(f"Setting max molecular weight: {max_mw:.1f} Da")

Pocket 1 volume: 406.7 Å³
Setting max molecular weight: 406.7 Da


In [ ]:
smiles = ligand_generator.generate_smiles()

Pocket center: -18.92,-2.99,13.69
Running Pocket2Mol...
Pocket2Mol completed successfully.


In [ ]:
import os
from rdkit import Chem
from rdkit.Chem import Descriptors, QED

def get_latest_smiles_file(base_dir=f"{base_dir}/Pocket2Mol/outputs"):
    # Find the newest timestamped directory
    subdirs = [
        os.path.join(base_dir, d)
        for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d))
    ]
    if not subdirs:
        raise FileNotFoundError("No output folders found in Pocket2Mol/outputs")

    latest_dir = max(subdirs, key=os.path.getmtime)
    smiles_path = os.path.join(latest_dir, "SMILES.txt")
    if not os.path.exists(smiles_path):
        raise FileNotFoundError(f"SMILES.txt not found in {latest_dir}")

    return smiles_path

def filter_smiles(file_path, top_n=5):
    with open(file_path) as f:
        smiles_list = [line.strip() for line in f if line.strip()]

    filtered = []

    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if not mol:
            continue

        mw = Descriptors.MolWt(mol)
        qed = QED.qed(mol)
        hba = Descriptors.NumHAcceptors(mol)
        hbd = Descriptors.NumHDonors(mol)
        logp = Descriptors.MolLogP(mol)

        lipinski = (hba <= 10) and (hbd <= 5) and (logp <= 5)
        if 100 < mw <= 600 and qed >= 0.4 and lipinski:
            filtered.append((smi, qed, mw))

    # Sort by QED score
    filtered.sort(key=lambda x: x[1], reverse=True)

    top_filtered = filtered[:top_n]
    print(f"Selected top {len(top_filtered)} ligands:")
    for smi, qed, mw in top_filtered:
        print(f"{smi} | QED: {qed:.2f} | MW: {mw:.2f}")

    return [smi for smi, _, _ in top_filtered]


In [ ]:
smiles_path = get_latest_smiles_file()
top_smiles = filter_smiles(smiles_path, top_n=5)

Selected top 5 ligands:
O=C(O)CCc1cccc(CO)c1 | QED: 0.73 | MW: 180.20
COc1ccc2c(O)cccc2c1 | QED: 0.72 | MW: 174.20
COC1CCc2c(O)cccc2C1 | QED: 0.71 | MW: 178.23
O=C(O)COc1ccccc1 | QED: 0.71 | MW: 152.15
O=C(O)c1cccc(C(=O)O)c1 | QED: 0.69 | MW: 166.13


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_sdf(smiles, sdf_path="molecule.sdf"):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())  # Generate 3D coordinates
    AllChem.UFFOptimizeMolecule(mol)  # Optional: minimize energy
    writer = Chem.SDWriter(sdf_path)
    writer.write(mol)
    writer.close()
    print(f"Saved SDF to: {sdf_path}")

smiles_to_sdf(top_smiles[0], "ligand_1.sdf")

Saved SDF to: ligand_1.sdf


In [ ]:
# smiles_to_sdf(top_smiles[0], "ligand_1.sdf")

### Evaluating the Ground State Energies

In [ ]:
!pip uninstall -y qiskit qiskit-nature pylatexenc rdkit --quiet
!pip install "qiskit==1.4.3" qiskit-nature qiskit-nature-pyscf pylatexenc rdkit --quiet

In [ ]:
def compute_molecule_coordinates(sdf_path):
    # Load the molecule from an SDF file
    supplier = Chem.SDMolSupplier(sdf_path, removeHs=False)
    mol = supplier[0]  # First molecule in the file

    if mol is None:
        raise ValueError("Could not read molecule from SDF. Check the file format.")

    # Generate 3D coordinates if not already present
    if not mol.GetConformer().Is3D():
        AllChem.EmbedMolecule(mol, AllChem.ETKDG())
        AllChem.MMFFOptimizeMolecule(mol)

    # Extract coordinates and format them
    conf = mol.GetConformer()
    coords = []

    for i in range(mol.GetNumAtoms()):
        atom = mol.GetAtomWithIdx(i)
        pos = conf.GetAtomPosition(i)
        coords.append(f"{atom.GetSymbol()} {pos.x:.6f} {pos.y:.6f} {pos.z:.6f}")

    formatted_coords = "; ".join(coords)

    return formatted_coords


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
molecule = "H 0 0 0; H 0 0 0.735" # H_2
# molecule = "N 0.000 0.000 0.100; H 0.9377 0.000 -0.270; H -0.46885 0.812 -0.270; H -0.46885 -0.812 -0.270" # NH_3
# molecule = "N 0.000 0.000 0.000; H 0.000 0.000 1.010"
## Load molecule automatically from SDF file that we can download from https://pubchem.ncbi.nlm.nih.gov/

## Eg: Pyrimethamine : https://pubchem.ncbi.nlm.nih.gov/compound/4993
# sdf_file = "Pyrimethamine_Conformer3D_COMPOUND_CID_4993.sdf"
# molecule = compute_molecule_coordinates(sdf_file)

print(molecule)

In [ ]:
driver = PySCFDriver(
    atom=molecule,
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [ ]:
ast = ActiveSpaceTransformer(num_electrons=2, num_molecular_orbitals=2)
es_problem = ast.transform(es_problem)

In [ ]:
mapper = JordanWignerMapper()

In [ ]:
fermionic_op = es_problem.hamiltonian.second_q_op()
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

In [ ]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [ ]:
ansatz.decompose().draw('mpl')

In [ ]:
calc = GroundStateEigensolver(mapper, vqe_solver)

In [ ]:
res = calc.solve(es_problem)

print(res)

In [ ]:
!pip uninstall -y qiskit qiskit-nature pylatexenc rdkit --quiet
!pip install "qiskit==1.4.3" qiskit-nature qiskit-nature-pyscf pylatexenc rdkit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.5 MB/s eta 0:00:00


In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit import QuantumCircuit
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from rdkit import Chem
from rdkit.Chem import AllChem
import time
import numpy as np

def smiles_to_geometry(smiles, basis="sto-3g", max_attempts=3):
    """Robust SMILES to geometry conversion with multiple optimization attempts"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError(f"Invalid SMILES: '{smiles}'")

        mol = Chem.AddHs(mol)
        for attempt in range(max_attempts):
            try:
                AllChem.EmbedMolecule(mol, randomSeed=42+attempt)
                AllChem.MMFFOptimizeMolecule(mol)
                break
            except:
                if attempt == max_attempts-1:
                    raise RuntimeError(f"Geometry optimization failed after {max_attempts} attempts")
                continue

        conf = mol.GetConformer()
        elements = []
        coords = []
        for i in range(mol.GetNumAtoms()):
            pos = conf.GetAtomPosition(i)
            elements.append(mol.GetAtomWithIdx(i).GetSymbol())
            coords.append([pos.x, pos.y, pos.z])

        # Format for PySCF: "Element x y z" for each atom
        geometry = "\n".join([f"{el} {x:.8f} {y:.8f} {z:.8f}"
                            for el, (x, y, z) in zip(elements, coords)])
        return geometry

    except Exception as e:
        raise RuntimeError(f"Geometry generation failed for {smiles}: {str(e)}")

def generate_molecular_circuit(smiles, basis="sto-3g", mapper_type="jordan_wigner", charge=0, spin=0):
    """
    Generate quantum circuit for molecular simulation with complete error handling

    Args:
        smiles (str): Molecular SMILES string
        basis (str): Quantum chemistry basis set
        mapper_type (str): 'jordan_wigner' or 'parity'
        charge (int): Molecular charge
        spin (int): Spin multiplicity (2S+1)

    Returns:
        dict: Contains circuit and metadata or error information
    """
    start_time = time.time()
    result = {
        "smiles": smiles,
        "basis": basis,
        "mapper": mapper_type,
        "charge": charge,
        "spin": spin,
        "success": False
    }

    try:
        # 1. Generate molecular geometry
        geometry = smiles_to_geometry(smiles, basis)
        result["geometry"] = geometry

        # 2. Run quantum chemistry calculation
        driver = PySCFDriver(
            atom=geometry,
            basis=basis,
            charge=charge,
            spin=spin-1,  # Convert multiplicity to 2S
            unit=DistanceUnit.ANGSTROM
        )
        problem = driver.run()

        # 3. Get molecule information
        molecule = problem.molecule
        result["molecule_info"] = {
            "n_electrons": molecule.num_alpha + molecule.num_beta,
            "n_alpha": molecule.num_alpha,
            "n_beta": molecule.num_beta,
            "n_orbitals": molecule.num_molecular_orbitals,
            "nuclear_repulsion": problem.nuclear_repulsion_energy,
            "atoms": [str(atom) for atom in molecule.atoms]
        }

        # 4. Select mapper
        mapper = JordanWignerMapper() if mapper_type == "jordan_wigner" else ParityMapper()

        # 5. Build UCCSD ansatz
        ansatz = UCCSD(
            molecule.num_molecular_orbitals,
            (molecule.num_alpha, molecule.num_beta),
            mapper,
            initial_state=HartreeFock(
                molecule.num_molecular_orbitals,
                (molecule.num_alpha, molecule.num_beta),
                mapper
            )
        )

        # 6. Prepare results
        result.update({
            "circuit": ansatz,
            "qubits": ansatz.num_qubits,
            "parameters": ansatz.num_parameters,
            "depth": ansatz.depth(),
            "success": True,
            "execution_time": time.time() - start_time
        })

    except Exception as e:
        result.update({
            "error": str(e),
            "execution_time": time.time() - start_time
        })

    return result

def save_circuit_for_qbraid(result, filename_prefix=""):
    """Save circuit in Qbraid-compatible format"""
    if not result["success"]:
        raise ValueError("Cannot save failed circuit generation")

    # Generate descriptive filename
    smiles_short = result["smiles"][:10].replace("/", "_")
    filename = f"{filename_prefix}{smiles_short}_q{result['qubits']}_p{result['parameters']}.qasm"

    # Save QASM file
    with open(filename, "w") as f:
        f.write(result["circuit"].qasm())

    # Save metadata as JSON
    import json
    with open(filename.replace(".qasm", ".json"), "w") as f:
        json.dump({
            k: v for k, v in result.items()
            if k not in ["circuit"]
        }, f, indent=2)

    return filename

# ====================
# Target Molecules
# ====================
MOLECULES = {
    "Succinic Acid": "O=C(O)CCc1cccc(CO)c1",
    "Guaicol Derivative": "COc1ccc2c(O)cccc2c1",
    "Methoxy Cyclohexadiene": "COC1CCc2c(O)cccc2C1",
    "Phenyl Formate": "O=C(O)COc1ccccc1",
    "Phthalic Acid": "O=C(O)c1cccc(C(=O)O)c1"
}

# ====================
# Execution
# ====================
if __name__ == "__main__":
    print("Generating quantum circuits for organic molecules...")
    print(f"{'Molecule':<25} {'Qubits':<8} {'Params':<8} {'Status':<15} {'Time (s)':<10}")
    print("-"*70)

    successful_circuits = {}

    for name, smiles in MOLECULES.items():
        # Generate circuit with error handling
        try:
            result = generate_molecular_circuit(
                smiles,
                basis="sto-3g",
                mapper_type="jordan_wigner",
                charge=0,
                spin=1
            )

            # Print status
            status = "SUCCESS" if result["success"] else "FAILED"
            time_str = f"{result['execution_time']:.1f}"
            qubits = result.get("qubits", "N/A")
            params = result.get("parameters", "N/A")

            print(f"{name[:24]:<25} {qubits:<8} {params:<8} {status:<15} {time_str:<10}")

            # Save successful circuits
            if result["success"]:
                filename = save_circuit_for_qbraid(result, f"{name[:3]}_")
                successful_circuits[name] = {
                    "filename": filename,
                    "qubits": result["qubits"],
                    "parameters": result["parameters"]
                }

        except Exception as e:
            print(f"{name[:24]:<25} {'-':<8} {'-':<8} {'CRASHED':<15} {'-':<10}")
            print(f"  Error: {str(e)}")

    # Print summary
    print("\nSuccessfully generated circuits:")
    for name, data in successful_circuits.items():
        print(f"- {name}: {data['filename']} ({data['qubits']} qubits, {data['parameters']} params)")

ModuleNotFoundError: No module named 'qiskit_nature'

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit import QuantumCircuit
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from rdkit import Chem
from rdkit.Chem import AllChem
import time
import numpy as np

def smiles_to_geometry(smiles, basis="sto-3g", max_attempts=3):
    """Robust SMILES to geometry conversion with multiple optimization attempts"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError(f"Invalid SMILES: '{smiles}'")

        mol = Chem.AddHs(mol)
        for attempt in range(max_attempts):
            try:
                AllChem.EmbedMolecule(mol, randomSeed=42+attempt)
                AllChem.MMFFOptimizeMolecule(mol)
                break
            except:
                if attempt == max_attempts-1:
                    raise RuntimeError(f"Geometry optimization failed after {max_attempts} attempts")
                continue

        conf = mol.GetConformer()
        elements = []
        coords = []
        for i in range(mol.GetNumAtoms()):
            pos = conf.GetAtomPosition(i)
            elements.append(mol.GetAtomWithIdx(i).GetSymbol())
            coords.append([pos.x, pos.y, pos.z])

        # Format for PySCF: "Element x y z" for each atom
        geometry = "\n".join([f"{el} {x:.8f} {y:.8f} {z:.8f}"
                            for el, (x, y, z) in zip(elements, coords)])
        return geometry

    except Exception as e:
        raise RuntimeError(f"Geometry generation failed for {smiles}: {str(e)}")

def generate_molecular_circuit(smiles, basis="sto-3g", mapper_type="jordan_wigner", charge=0, spin=0):
    """
    Generate quantum circuit for molecular simulation with complete error handling

    Args:
        smiles (str): Molecular SMILES string
        basis (str): Quantum chemistry basis set
        mapper_type (str): 'jordan_wigner' or 'parity'
        charge (int): Molecular charge
        spin (int): Spin multiplicity (2S+1)

    Returns:
        dict: Contains circuit and metadata or error information
    """
    start_time = time.time()
    result = {
        "smiles": smiles,
        "basis": basis,
        "mapper": mapper_type,
        "charge": charge,
        "spin": spin,
        "success": False
    }

    try:
        # 1. Generate molecular geometry
        geometry = smiles_to_geometry(smiles, basis)
        result["geometry"] = geometry

        # 2. Run quantum chemistry calculation
        driver = PySCFDriver(
            atom=geometry,
            basis=basis,
            charge=charge,
            spin=spin-1,  # Convert multiplicity to 2S
            unit=DistanceUnit.ANGSTROM
        )
        problem = driver.run()

        # 3. Get molecule information (FIXED ATTRIBUTE ACCESS)
        num_alpha = problem.num_alpha_electrons
        num_beta = problem.num_beta_electrons
        num_orbitals = problem.num_spatial_orbitals

        result["molecule_info"] = {
            "n_electrons": num_alpha + num_beta,
            "n_alpha": num_alpha,
            "n_beta": num_beta,
            "n_orbitals": num_orbitals,
            "nuclear_repulsion": problem.nuclear_repulsion_energy,
            "atoms": [str(atom) for atom in problem.molecule.atoms]
        }

        # 4. Select mapper
        mapper = JordanWignerMapper() if mapper_type == "jordan_wigner" else ParityMapper()

        # 5. Build UCCSD ansatz (USING FIXED ATTRIBUTES)
        ansatz = UCCSD(
            num_orbitals,
            (num_alpha, num_beta),
            mapper,
            initial_state=HartreeFock(
                num_orbitals,
                (num_alpha, num_beta),
                mapper
            )
        )

        # 6. Prepare results
        result.update({
            "circuit": ansatz,
            "qubits": ansatz.num_qubits,
            "parameters": ansatz.num_parameters,
            "depth": ansatz.depth(),
            "success": True,
            "execution_time": time.time() - start_time
        })

    except Exception as e:
        result.update({
            "error": str(e),
            "execution_time": time.time() - start_time
        })

    return result


def save_circuit_for_qbraid(result, filename_prefix=""):
    """Save circuit in Qbraid-compatible format"""
    if not result["success"]:
        raise ValueError("Cannot save failed circuit generation")

    # Generate descriptive filename
    smiles_short = result["smiles"][:10].replace("/", "_")
    filename = f"{filename_prefix}{smiles_short}_q{result['qubits']}_p{result['parameters']}.qasm"

    # Save QASM file
    with open(filename, "w") as f:
        f.write(result["circuit"].qasm())

    # Save metadata as JSON
    import json
    with open(filename.replace(".qasm", ".json"), "w") as f:
        json.dump({
            k: v for k, v in result.items()
            if k not in ["circuit"]
        }, f, indent=2)

    return filename

# ====================
# Target Molecules
# ====================
MOLECULES = {
    "Succinic Acid": "O=C(O)CCc1cccc(CO)c1",
    "Guaicol Derivative": "COc1ccc2c(O)cccc2c1",
    "Methoxy Cyclohexadiene": "COC1CCc2c(O)cccc2C1",
    "Phenyl Formate": "O=C(O)COc1ccccc1",
    "Phthalic Acid": "O=C(O)c1cccc(C(=O)O)c1"
}

# ====================
# Execution
# ====================
if __name__ == "__main__":
    print("Generating quantum circuits for organic molecules...")
    print(f"{'Molecule':<25} {'Qubits':<8} {'Params':<8} {'Status':<15} {'Time (s)':<10}")
    print("-"*70)

    successful_circuits = {}

    for name, smiles in MOLECULES.items():
        # Generate circuit with error handling
        try:
            result = generate_molecular_circuit(
                smiles,
                basis="sto-3g",
                mapper_type="jordan_wigner",
                charge=0,
                spin=1
            )

            # Print status
            status = "SUCCESS" if result["success"] else "FAILED"
            time_str = f"{result['execution_time']:.1f}"
            qubits = result.get("qubits", "N/A")
            params = result.get("parameters", "N/A")

            print(f"{name[:24]:<25} {qubits:<8} {params:<8} {status:<15} {time_str:<10}")

            # Save successful circuits
            if result["success"]:
                filename = save_circuit_for_qbraid(result, f"{name[:3]}_")
                successful_circuits[name] = {
                    "filename": filename,
                    "qubits": result["qubits"],
                    "parameters": result["parameters"]
                }

        except Exception as e:
            print(f"{name[:24]:<25} {'-':<8} {'-':<8} {'CRASHED':<15} {'-':<10}")
            print(f"  Error: {str(e)}")

    # Print summary
    print("\nSuccessfully generated circuits:")
    for name, data in successful_circuits.items():
        print(f"- {name}: {data['filename']} ({data['qubits']} qubits, {data['parameters']} params)")

Generating quantum circuits for organic molecules...
Molecule                  Qubits   Params   Status          Time (s)  
----------------------------------------------------------------------
Succinic Acid             N/A      N/A      FAILED          13.4      
Guaicol Derivative        N/A      N/A      FAILED          13.3      
Methoxy Cyclohexadiene    N/A      N/A      FAILED          18.4      
Phenyl Formate            N/A      N/A      FAILED          7.3       
Phthalic Acid             N/A      N/A      FAILED          8.3       

Successfully generated circuits:


In [ ]:
molecules = [
    {'name': 'H2', 'geometry': 'H .0 .0 .0; H .0 .0 0.735'},
    {'name': 'LiH', 'geometry': 'Li .0 .0 .0; H .0 .0 1.6'},
    {'name': 'H2O', 'geometry': 'O .0 .0 .0; H .0 .757 .586; H .0 -.757 .586'}
]


ModuleNotFoundError: No module named 'qiskit_nature'

## 3. List of target molecules (SMILES) and run

In [ ]:
smiles_list = [
    "O=C(O)CCc1cccc(CO)c1",
    "COc1ccc2c(O)cccc2c1",
    "COC1CCc2c(O)cccc2C1",
    "O=C(O)COc1ccccc1",
    "O=C(O)c1cccc(C(=O)O)c1"]
for smi in smiles_list:
    geo = smiles_to_geometry(smi)
    print(f"Running VQE for {smi} …")
    energy = run_vqe(geo)
    print(f" → Estimated ground state energy: {energy:.6f} Hartree\n")


NameError: name 'smiles_to_geometry' is not defined